#**1: Install All the Required Packages**

In [1]:
!pip -q install langchain
!pip -q install bitsandbytes accelerate transformers
!pip -q install datasets loralib sentencepiece
!pip -q install pypdf
!pip -q install sentence_transformers
!pip install langchain-community
!pip -q install unstructured
!pip install tokenizers
!pip install xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [16]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 58.3 MB/s eta 0:00:00


#**2: Import All the Required Libraries**

In [2]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain import HuggingFacePipeline

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

from huggingface_hub import notebook_login
import textwrap
import sys
import os
import torch


### Login

In [3]:
notebook_login()

In [4]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

#**3: Pass the URLs and extract the data from these URLs**

In [5]:
URLs=[
    'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb',
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models',
    'https://lmsys.org/blog/2023-03-30-vicuna/'

]

In [6]:
loaders = UnstructuredURLLoader(urls=URLs)
data = loaders.load()

In [7]:
data

[Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Open in app\n\nSign up\n\nSign in\n\nWrite\n\nSign up\n\nSign in\n\nPaper Review\n\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\n\nLlama 2: one of the best open source models\n\nAndrew Lukyanenko\n\nFollow\n\nPublished in\n\nGoPenAI\n\n15 min read\n\nJul 20, 2023\n\n--\n\nProject link\n\nModel link\n\nPaper link\n\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, they could potentially replace closed-source models. The authors also detail their ap

In [8]:
len(data)

4

#**4: Split the Text into Chunks**

In [9]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [10]:
text_chunks = text_splitter.split_documents(data)

In [11]:
len(text_chunks)

86

#**5: Download the Hugging Face Embeddings**

In [12]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

<ipython-input-12-8cc2085ef130>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or dat

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
query_result = embeddings.embed_query("How are you")
len(query_result)

384

In [14]:
query_result

[-0.03136897832155228,
 0.03783053532242775,
 0.07630816847085953,
 0.04569968581199646,
 -0.0012047342024743557,
 -0.0747690498828888,
 0.08157844841480255,
 0.010209362022578716,
 -0.11220554262399673,
 0.04073434695601463,
 -0.04470574110746384,
 -0.009028705768287182,
 -0.022976206615567207,
 -0.00918252021074295,
 0.007133581209927797,
 -0.0352972075343132,
 0.07896503061056137,
 -0.09915590286254883,
 -0.1210382804274559,
 0.03257369622588158,
 -0.0991876944899559,
 0.03196954354643822,
 0.0017394950846210122,
 0.08815880119800568,
 -0.02667001634836197,
 0.01709165796637535,
 -0.04135075584053993,
 -0.039181794971227646,
 0.03500916808843613,
 -0.07608218491077423,
 -0.06472425162792206,
 0.022676298394799232,
 -0.04956904798746109,
 -0.027594150975346565,
 -0.057269610464572906,
 -0.04053007811307907,
 0.016540292650461197,
 -0.10059664398431778,
 -0.04873242229223251,
 -0.022784549742937088,
 0.020855015143752098,
 -0.06302400678396225,
 -0.019444210454821587,
 -0.027309581637

#**6: Convert the Text Chunks into Embeddings and Create a Knowledge Base**

In [17]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)

#**7: Create a Large Language Model (LLM) Wrapper**

In [18]:
model = "meta-llama/Llama-2-7b-chat-hf"

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model,
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained(model,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              load_in_8bit=True,
                                              #load_in_4bit=True
                                             )

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

Device set to use cuda:0


In [22]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

<ipython-input-22-9753c688c802>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})


In [23]:
llm.predict("Please provide a concise summary of the Book Harry Potter")

<ipython-input-23-181e585c8cae>:1: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict("Please provide a concise summary of the Book Harry Potter")
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"Please provide a concise summary of the Book Harry Potter and the Philosopher's Stone by J.K. Rowling.\n\nBook Summary:\nHarry Potter is an orphan who lives with his cruel and neglectful relatives, the Dursleys. On his eleventh birthday, he receives a letter informing him that he is a wizard and that he will be attending Hogwarts School of Witchcraft and Wizardry. Harry soon discovers that he is famous in the wizarding world, as his parents were killed by the dark wizard, Lord Voldemort. Voldemort, also known as He-Who-Must-Not-Be-Named, tried to kill Harry when he was a baby, but he failed and was forced to flee. Harry becomes fast friends with Ron Weasley and Hermione Granger, and together they embark on a series of adventures at Hogwarts, including uncovering a mysterious object known as the Philosopher's Stone. Along the way, Harry and his friends must contend with bullies, tricky potions, and a series of obstacles that test their courage and loyalty to each other. In the end, Har

#**8: Initialize the Retrieval QA with Source Chain**

In [24]:
from langchain.chains import RetrievalQA

In [25]:
query = "How good is Vicuna?"

In [26]:
docs = vectorstore.similarity_search(query, k=3)

In [28]:
# Extract only results
results = []
for doc in docs:
    results.append(doc.page_content)

# Join and print results
final_result = "\n\n".join(results)
print(final_result)


LMSYS ORG
ProjectsBlogAboutDonationsChatbot Arena
Vicuna: An Open-Source Chatbot Impressing GPT-4 with 90%* ChatGPT Quality
by: The Vicuna Team, Mar 30, 2023
We introduce Vicuna-13B, an open-source chatbot trained by fine-tuning LLaMA on user-shared conversations collected from ShareGPT. Preliminary evaluation using GPT-4 as a judge shows Vicuna-13B achieves more than 90%* quality of OpenAI ChatGPT and Google Bard while outperforming other models like LLaMA and Stanford Alpaca in more than 90%* of cases. The cost of training Vicuna-13B is around $300. The code and weights, along with an online demo, are publicly available for non-commercial use.
Vicuna (generated by stable diffusion 2.1)
According to a fun and non-scientific evaluation with GPT-4. Further rigorous evaluation is needed.
How Good is Vicuna?

Vicuna (generated by stable diffusion 2.1)
According to a fun and non-scientific evaluation with GPT-4. Further rigorous evaluation is needed.
How Good is Vicuna?
After fine-tuning V

In [29]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

In [37]:
query = "How good is Vicuna?"
qa.run(query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nLMSYS ORG\nProjectsBlogAboutDonationsChatbot Arena\nVicuna: An Open-Source Chatbot Impressing GPT-4 with 90%* ChatGPT Quality\nby: The Vicuna Team, Mar 30, 2023\nWe introduce Vicuna-13B, an open-source chatbot trained by fine-tuning LLaMA on user-shared conversations collected from ShareGPT. Preliminary evaluation using GPT-4 as a judge shows Vicuna-13B achieves more than 90%* quality of OpenAI ChatGPT and Google Bard while outperforming other models like LLaMA and Stanford Alpaca in more than 90%* of cases. The cost of training Vicuna-13B is around $300. The code and weights, along with an online demo, are publicly available for non-commercial use.\nVicuna (generated by stable diffusion 2.1)\nAccording to a fun and non-scientific evaluation with GPT-4. Further rigorous evaluation is needed.\nHow Good is Vicuna?\n\nVicuna

In [36]:
query = "How does Llama 2 outperforms other models"
qa.run(query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nPretraining Data\nThe authors utilized a novel mix of data from publicly accessible sources to train the Llama 2 models, excluding any data from Meta’s products or services. They made efforts to erase data from certain sites known for harboring large amounts of personal information about private individuals. They trained the models on 2 trillion tokens of data, believing this amount provided a beneficial performance-cost balance. They also up-sampled the most factual sources to boost knowledge and reduce instances of false information generation or “hallucinations”.\nLlama 2 Pretrained Model Evaluation\nLlama 2 models significantly outperform their Llama 1 counterparts:\nThe 70 billion-parameter Llama 2 model notably improves results on the MMLU and BBH benchmarks by roughly 5 and 8 points, respectively, when compared to 

In [38]:
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Input Prompt: How does Llama 2 outperforms other model


<ipython-input-38-c213a85929f6>:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({'query': user_input})
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Pretraining Data
The authors utilized a novel mix of data from publicly accessible sources to train the Llama 2 models, excluding any data from Meta’s products or services. They made efforts to erase data from certain sites known for harboring large amounts of personal information about private individuals. They trained the models on 2 trillion tokens of data, believing this amount provided a beneficial performance-cost balance. They also up-sampled the most factual sources to boost knowledge and reduce instances of false information generation or “hallucinations”.
Llama 2 Pretrained Model Evaluation
Llama 2 models significantly outperform their Llama 1 counterparts:
The 70 billion-parameter Llama 2 model notably improves results on the MMLU and BBH benchmarks by roughly 5 and 8 points, respectively, when compared to

SystemExit: 

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
